In [ ]:
 !conda install -y gdown

In [ ]:
# # Model Weights
# !gdown --id 1-0WEEnoyfgrZm1QL-1jdZ6wbhBpymuX_

In [1]:
# # Actuall data
!gdown --id 1COfReFvSiXkiZjX5-rFgNjLHfjLj2MFF

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


/bin/bash: gdown: command not found


In [ ]:
import pandas as pd
import numpy as np
import SimpleITK as sitk
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import tensorflow as tf
import nibabel as nib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
import pickle
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import warnings
warnings.filterwarnings("ignore")

In [ ]:
file_path = '/kaggle/working/All_images_tuple_32256256.pkl'

with open(file_path, 'rb') as file:
    train_images, train_labels,val_images, val_labels = pickle.load(file)

In [ ]:
for_file = '32256256'
for_model = (32 ,256 ,256, 1)
for_processing = (32 ,256 ,256)
#load_name = 'model_checkpoint_256_V2_300_epochs_full_unet256'
store_name = 'model_checkpoint_256_V2_100_epochs_full_unet256_1024_3_layers'

In [ ]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Conv3DTranspose, concatenate

def conv_block(input_tensor, num_filters):
    x = Conv3D(num_filters, (3, 3, 3), activation='relu', padding='same')(input_tensor)
    x = Conv3D(num_filters, (3, 3, 3), activation='relu', padding='same')(x)
    return x

def encoder_block(input_tensor, num_filters):
    x = conv_block(input_tensor, num_filters)
    p = MaxPooling3D((2, 2, 2))(x)
    return x, p

def decoder_block(input_tensor, concat_tensor, num_filters):
    x = Conv3DTranspose(num_filters, (2, 2, 2), strides=(2, 2, 2), padding='same')(input_tensor)
    x = concatenate([x, concat_tensor], axis=-1)
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape, n_classes):
    inputs = Input(input_shape)

    # Encoder
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    #s4, p4 = encoder_block(p3, 512)

    # Bridge
    b1 = conv_block(p3, 1024)

    # Decoder
    #d1 = decoder_block(b1, s4, 512)
    d1 = decoder_block(b1, s3, 256)
    d2 = decoder_block(d1, s2, 128)
    d3 = decoder_block(d2, s1, 64)

    # Output layer
    if n_classes == 1:
        activation = 'sigmoid'
    else:
        activation = 'softmax'
    outputs = Conv3D(n_classes, (1, 1, 1), padding="same", activation=activation)(d3)

    model = Model(inputs=[inputs], outputs=[outputs], name="3D-U-Net")

    return model

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    return tf.keras.losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

# Create the model
input_shape = for_model
n_classes = 1
model = build_unet(input_shape, n_classes)

model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[dice_coefficient])

model.summary()

In [ ]:
print(store_name)

In [ ]:
# file_path = f'/kaggle/working/{load_name}.keras'
# model.load_weights(file_path)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

model_checkpoint_path = f'/kaggle/working/{store_name}.keras'
callbacks = [
    ModelCheckpoint(model_checkpoint_path, save_best_only=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20),
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
]

history = model.fit(
    train_images, train_labels,
    epochs=100,
    batch_size=1,
    validation_data=(val_images, val_labels),
    callbacks = callbacks
)

In [ ]:
# model.save(f'{store_name}.keras')

In [ ]:
file_path = f'/kaggle/working/{store_name}.keras'
model.load_weights(file_path)

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K

num_images = 3
test_indices = np.random.choice(np.arange(val_images.shape[0]), size=num_images, replace=False)

test_predictions = model.predict(val_images[test_indices])

for i, idx in enumerate(test_indices):
    img, true_mask, pred_mask = val_images[idx], val_labels[idx], test_predictions[i]

    x_slice_idx = img.shape[0] // 2
    y_slice_idx = img.shape[1] // 2
    z_slice_idx = img.shape[2] // 2

    plt.figure(figsize=(18, 6))

    plt.subplot(1, 3, 1)
    plt.imshow(img[x_slice_idx, :, :, 0], cmap='gray')
    plt.title("MRI Input (X-axis)")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(true_mask[x_slice_idx, :, :, 0], cmap='gray')
    plt.title("True Label (X-axis)")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(pred_mask[x_slice_idx, :, :, 0], cmap='gray')
    plt.title("Predicted Label (X-axis)")
    plt.axis('off')

    plt.show()

    dice_score_x = dice_coefficient(true_mask[x_slice_idx], pred_mask[x_slice_idx]).numpy()
    print(f"Dice Similarity Score (X-axis): {dice_score_x}")

    plt.figure(figsize=(18, 6))

    plt.subplot(1, 3, 1)
    plt.imshow(img[:, y_slice_idx, :, 0], cmap='gray')
    plt.title("MRI Input (Y-axis)")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(true_mask[:, y_slice_idx, :, 0], cmap='gray')
    plt.title("True Label (Y-axis)")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(pred_mask[:, y_slice_idx, :, 0], cmap='gray')
    plt.title("Predicted Label (Y-axis)")
    plt.axis('off')

    plt.show()

    dice_score_y = dice_coefficient(true_mask[:, y_slice_idx], pred_mask[:, y_slice_idx]).numpy()
    print(f"Dice Similarity Score (Y-axis): {dice_score_y}")

    plt.figure(figsize=(18, 6))

    plt.subplot(1, 3, 1)
    plt.imshow(img[:, :, z_slice_idx, 0], cmap='gray')
    plt.title("MRI Input (Z-axis)")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(true_mask[:, :, z_slice_idx, 0], cmap='gray')
    plt.title("True Label (Z-axis)")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(pred_mask[:, :, z_slice_idx, 0], cmap='gray')
    plt.title("Predicted Label (Z-axis)")
    plt.axis('off')

    plt.show()

    dice_score_z = dice_coefficient(true_mask[:, :, z_slice_idx], pred_mask[:, :, z_slice_idx]).numpy()
    print(f"Dice Similarity Score (Z-axis): {dice_score_z}")
